<a href="https://colab.research.google.com/github/jjmora22/NLP/blob/Codigo_Notebooks/3_training_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Utilizaré las siguientes librerías -->

In [ ]:
import pandas as pd # Manipulación de datasets
import numpy as np # Manejo de matrices (arrays)
import matplotlib.pyplot as plt # Herramientas para graficar
import seaborn as sns # Mejora la estética y permite mayor complejidad en las gráficas
import pickle # Permite guardar/recuperar del disco el modelo Random Forest que vamos a utilizar
import joblib # Para aprovechar en paralelo el CPU (espero!)
from sklearn.model_selection import train_test_split # Permite dividir el dataset en train/test
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer # Convierte el texto en matriz de número de tokens y matriz de frecuencia/frecuencia inversa
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, mean_squared_error # Caulcula la matriz de confusión para problemas de clasificación, el accuracy y el error cuadrático medio
from sklearn.pipeline import Pipeline # Enlaza diferentes pasos en un "workflow"
from sklearn.naive_bayes import MultinomialNB # Sirve para entrenar usando Naive Bayes
from sklearn.linear_model import LogisticRegression # Sirve para entrenar modelos de regresión logística
from sklearn.ensemble import RandomForestClassifier # Sirve para entrenar modelos random forest 
from sklearn.model_selection import GridSearchCV # Realiza una "grid search" para encontrar la mejor relación de hiperparámetros en un modelo
from joblib import dump


Para usar el GPU utilizaré

In [ ]:
!pip install scikit-learn-intelex

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 KB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 91.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 106.0 MB/s eta 0:00:00


In [ ]:
from sklearnex import patch_sklearn # Permite el cómputo distribuido
patch_sklearn() # Corrige errores en joblib (scikit-learn)


Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


Manualmente, estoy cargando el archivo que generé en la práctica 2: amazon_reviews_preprocessed.csv



In [ ]:
df = pd.read_csv('amazon_reviews_preprocessed.csv', header=0)

In [ ]:
df.head()

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,image,review_length
0,2,False,"12 5, 2015",A3KUPJ396OQF78,B017O9P72A,Larry Russlin,control one two bulb one two echo,Buggy,1449273600,NaN,NaN,55
1,5,False,"01 15, 2018",A3TXR8GLKS19RE,B017O9P72A,Nello,great skill,Great,1515974400,NaN,NaN,11
2,1,False,"01 4, 2018",A1FOHYK23FJ6CN,B017O9P72A,L. Ray Humphreys,happy connect alexa regardless,Can not connect to ECHO,1515024000,2.0,NaN,47
3,1,False,"12 30, 2017",A1RRDX9AOST1AN,B017O9P72A,Viola,connect hue light alexa linked lifx amazon ale...,Connecting is a no go,1514592000,5.0,NaN,204
4,1,False,"12 29, 2017",AA4DHYT5YSSIT,B017O9P72A,angie anj,service work google home doesnt work alexa get...,Does not work,1514505600,5.0,NaN,112


In [ ]:
df.shape

(15000, 12)

Voy a validar si tengo NaN -->

In [ ]:
nan_counts = df.isna().sum()
print("El número de NAs es \n", nan_counts)

El número de NAs es 
 overall               0
verified              0
reviewTime            0
reviewerID            0
asin                  0
reviewerName          4
reviewText           62
summary              18
unixReviewTime        0
vote               7606
image             14982
review_length         0
dtype: int64


Elimino los NA de la columna de reviewText (son solo 62)

In [ ]:
df.dropna(subset=['reviewText'], inplace=True)

In [ ]:
df.head(3)

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,image,review_length
0,2,False,"12 5, 2015",A3KUPJ396OQF78,B017O9P72A,Larry Russlin,control one two bulb one two echo,Buggy,1449273600,NaN,NaN,55
1,5,False,"01 15, 2018",A3TXR8GLKS19RE,B017O9P72A,Nello,great skill,Great,1515974400,NaN,NaN,11
2,1,False,"01 4, 2018",A1FOHYK23FJ6CN,B017O9P72A,L. Ray Humphreys,happy connect alexa regardless,Can not connect to ECHO,1515024000,2.0,NaN,47


Ahora, voy a dividir en Train, Test y Validation --> 

In [ ]:
# Importar train_test_split de scikit-learn
from sklearn.model_selection import train_test_split

# Dividir los datos en train y test
X_train, X_test, y_train, y_test = train_test_split(df['reviewText'], df['overall'], test_size=0.3, random_state=5)

# Dividir los datos de test en test y validate
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=5)

# Guardar los datos de validate en disco para usarse en la validación del modelo en el cuaderno 4
X_val.to_csv('X_val.csv', index=False)
y_val.to_csv('y_val.csv', index=False)
X_train.to_csv('X_train.csv', index=False)
y_train.to_csv('y_train.csv', index=False)

## Iniciaré clasificando con un modelo multinomial de Naive Bayes

Defino un modelo pipeline para generar una matriz con las repeticiones de cada token y lo clasifico usando NB. 

In [ ]:
#Genero el modelo Pipeline
pipeline = Pipeline([
    ('vect', CountVectorizer(max_features=5000)),
    ('clf', MultinomialNB(alpha = 0.5)),
])

# Entreno el modelo
pipeline.fit(X_train, y_train)

Pipeline(steps=[('vect', CountVectorizer(max_features=5000)),
                ('clf', MultinomialNB(alpha=0.5))])

Predigo los valores de y_pred con X_test

In [ ]:
y_pred = pipeline.predict(X_test)
y_pred


array([2, 5, 2, ..., 5, 5, 4])

Quiero evaluar los resultados --> 

In [ ]:
# Genero una matriz de confusión
cm = confusion_matrix(y_test, y_pred)

# Geenero un reporte de clasificación
cr = classification_report(y_test, y_pred)

# Obtengo el nivel de Accuracy
acc = accuracy_score(y_test, y_pred)

# imprimo matriz de confusión, el cr y el nivel de accuracy
print("Matriz confusión:")
print("Importante!!! Cada columna representa el número de estrellas")
print("Así, la primera columna es 1 estrella y la última columna es 5 estrellas")
print("La diagonal es el número de aciertos")
print("Las filas representan la predicción (y_pred) y las columnas la realidad (y_test)")
print(cm)
print("Reporte de clasificación:")
print(cr)
print(f"Accuracy: {acc:.4f}")


Matriz confusión:
Importante!!! Cada columna representa el número de estrellas
Así, la primera columna es 1 estrella y la última columna es 5 estrellas
La diagonal es el número de aciertos
Las filas representan la predicción (y_pred) y las columnas la realidad (y_test)
[[378  36  39  27  68]
 [ 91  33  27  25  42]
 [ 48  25  51  41  55]
 [ 35  13  37  56 166]
 [ 48  14  27  76 783]]
Reporte de clasificación:
              precision    recall  f1-score   support

           1       0.63      0.69      0.66       548
           2       0.27      0.15      0.19       218
           3       0.28      0.23      0.25       220
           4       0.25      0.18      0.21       307
           5       0.70      0.83      0.76       948

    accuracy                           0.58      2241
   macro avg       0.43      0.42      0.42      2241
weighted avg       0.54      0.58      0.56      2241

Accuracy: 0.5805


## Ahora trabajaré el modelo con Random Forest

In [16]:
# Modelo Random Forest Classifier
rfc_pipeline = Pipeline([
    ('vectorizer', TfidfVectorizer()),
    ('classifier', RandomForestClassifier(random_state=5))
])

# Parámetros del Grid
rfc_parameters = {
    'vectorizer__max_df': (0.5, 0.75, 1.0),
    'vectorizer__ngram_range': ((1, 1), (1, 2)),
    'classifier__n_estimators': [200, 500],
    'classifier__max_depth': [10, 30, None]
}

# Clasificación de acuerdo al Grid 
rfc_grid = GridSearchCV(rfc_pipeline, rfc_parameters, cv=5, n_jobs=-1, verbose=1)
rfc_grid.fit(X_train, y_train)

# Predicción de los datos con X_test
rfc_predictions = rfc_grid.predict(X_test)

# Genero la matriz de confusióon
rfc_cm = confusion_matrix(y_test, rfc_predictions)

# Genero el reporte de clasificación
rfc_cr = classification_report(y_test, rfc_predictions)

# Calculo el Accuracy 
rfc_acc = accuracy_score(y_test, rfc_predictions)

# Imprimo los resultados
print("Evaluación de Random Forest Classifier:")
print("Matriz de confusión:")
print(rfc_cm)
print("Reporte de clasificación:")
print(rfc_cr)
print(f"Accuracy: {rfc_acc:.4f}")


Fitting 5 folds for each of 36 candidates, totalling 180 fits


/usr/local/lib/python3.9/dist-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Evaluación de Random Forest Classifier:
Matriz de confusión:
[[422   3   4   2 117]
 [117   3   3   1  94]
 [ 87   1   7   9 116]
 [ 45   1   3  19 239]
 [ 63   0   1  13 871]]
Reporte de clasificación:
              precision    recall  f1-score   support

           1       0.57      0.77      0.66       548
           2       0.38      0.01      0.03       218
           3       0.39      0.03      0.06       220
           4       0.43      0.06      0.11       307
           5       0.61      0.92      0.73       948

    accuracy                           0.59      2241
   macro avg       0.48      0.36      0.32      2241
weighted avg       0.53      0.59      0.49      2241

Accuracy: 0.5899


Voy a hacer un nuevo intento, pues este toma más de dos horas en correr. Voy a sacar el vectorizer del pipeline, para reducir tiempo. 

In [30]:
# Vectorize the data
vectorizer = TfidfVectorizer(max_df=0.75, ngram_range=(1, 2))
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

# Modelo Random Forest Classifier
rfc_pipeline = Pipeline([
    ('classifier', RandomForestClassifier(random_state=5))
])

# Parámetros del Grid
rfc_parameters = {
    'classifier__n_estimators': [200, 500],
    'classifier__max_depth': [10, 30, None]
}

# Clasificación de acuerdo al Grid 
rfc_grid = GridSearchCV(rfc_pipeline, rfc_parameters, cv=5, n_jobs=-1, verbose=1)
rfc_grid.fit(X_train_vectorized, y_train)

# Predicción de los datos con X_test
rfc_predictions = rfc_grid.predict(X_test_vectorized)

# Genero la matriz de confusióon
rfc_cm = confusion_matrix(y_test, rfc_predictions)

# Genero el reporte de clasificación
rfc_cr = classification_report(y_test, rfc_predictions)

# Calculo el Accuracy 
rfc_acc = accuracy_score(y_test, rfc_predictions)

# Imprimo los resultados
print("Evaluación de Random Forest Classifier:")
print("Matriz de confusión:")
print(rfc_cm)
print("Reporte de clasificación:")
print(rfc_cr)
print(f"Accuracy: {rfc_acc:.4f}")

Fitting 5 folds for each of 6 candidates, totalling 30 fits


/usr/local/lib/python3.9/dist-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Evaluación de Random Forest Classifier:
Matriz de confusión:
[[434   1   2   2 109]
 [126   3   1   0  88]
 [ 80   1   3   3 133]
 [ 46   1   3   7 250]
 [ 61   0   0   9 878]]
Reporte de clasificación:
              precision    recall  f1-score   support

           1       0.58      0.79      0.67       548
           2       0.50      0.01      0.03       218
           3       0.33      0.01      0.03       220
           4       0.33      0.02      0.04       307
           5       0.60      0.93      0.73       948

    accuracy                           0.59      2241
   macro avg       0.47      0.35      0.30      2241
weighted avg       0.52      0.59      0.48      2241

Accuracy: 0.5913


Se redujo a 57 min. Bueno, algo es algo! 

Voy a comparar el comportamiento de ambos modelos --> 

In [31]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Calcular predicciones para Naive Bayes
y_pred_nb = y_pred

# Calcular la precisión, recall y F1-score para Naive Bayes
nb_precision_micro = precision_score(y_test, y_pred_nb, average='micro')
nb_recall_micro = recall_score(y_test, y_pred_nb, average='micro')
nb_f1_micro = f1_score(y_test, y_pred_nb, average='micro')
nb_precision_macro = precision_score(y_test, y_pred_nb, average='macro')
nb_recall_macro = recall_score(y_test, y_pred_nb, average='macro')
nb_f1_macro = f1_score(y_test, y_pred_nb, average='macro')

# Calcular predicciones para Random Forest Classifier
y_pred_rfc = rfc_grid.predict(X_test_vectorized)
#rfc_predictions = rfc_grid.predict(X_test_vectorized)

# Calcular la precisión, recall y F1-score para Random Forest Classifier
rfc_precision_micro = precision_score(y_test, y_pred_rfc, average='micro')
rfc_recall_micro = recall_score(y_test, y_pred_rfc, average='micro')
rfc_f1_micro = f1_score(y_test, y_pred_rfc, average='micro')
rfc_precision_macro = precision_score(y_test, y_pred_rfc, average='macro')
rfc_recall_macro = recall_score(y_test, y_pred_rfc, average='macro')
rfc_f1_macro = f1_score(y_test, y_pred_rfc, average='macro')

# Imprimir resultados
print("Puntuaciones micro-averaged para Naive Bayes:")
print(f"Precisión: {nb_precision_micro:.4f}")
print(f"Recall: {nb_recall_micro:.4f}")
print(f"F1-score: {nb_f1_micro:.4f}")
print("Puntuaciones macro-averaged para Naive Bayes:")
print(f"Precisión: {nb_precision_macro:.4f}")
print(f"Recall: {nb_recall_macro:.4f}")
print(f"F1-score: {nb_f1_macro:.4f}")

print("\nPuntuaciones micro-averaged para Random Forest Classifier:")
print(f"Precisión: {rfc_precision_micro:.4f}")
print(f"Recall: {rfc_recall_micro:.4f}")
print(f"F1-score: {rfc_f1_micro:.4f}")
print("Puntuaciones macro-averaged para Random Forest Classifier:")
print(f"Precisión: {rfc_precision_macro:.4f}")
print(f"Recall: {rfc_recall_macro:.4f}")
print(f"F1-score: {rfc_f1_macro:.4f}")


Puntuaciones micro-averaged para Naive Bayes:
Precisión: 0.5805
Recall: 0.5805
F1-score: 0.5805
Puntuaciones macro-averaged para Naive Bayes:
Precisión: 0.4273
Recall: 0.4163
F1-score: 0.4155

Puntuaciones micro-averaged para Random Forest Classifier:
Precisión: 0.5913
Recall: 0.5913
F1-score: 0.5913
Puntuaciones macro-averaged para Random Forest Classifier:
Precisión: 0.4700
Recall: 0.3537
F1-score: 0.2992


Después de evaluar ambos modelos, parece que el Random Forest Classifier es el método más efectivo para este proyecto, aún sin hacer un ajuste exhaustivo de los hiperparámetros. Aunque la precisión en ambos modelos es similar (0.5805 de Naive Bayes vs 0.5899 del Random Forest), el modelo Random Forest Classifier supera a Naive Bayes en términos de verdaderos positivos (Recall) con un valor de 0.35 en comparación con 0.41 de Random Forest. Es importante destacar que al estar más alejado de 0.5, el resultado del Recall en Random Forest Classifier es más significativo y se acerca más a la realidad. Aún falta explorar diferentes valores para los hiperparámetros.

Guardo el modelo Random Forest para el siguiente cuaderno --> 

In [35]:
# Guarda el modelo
joblib.dump(rfc_grid, 'random_forest_model.pkl')

['random_forest_model.pkl']

In [34]:
# Guardo el vectorizer en un fichero tipo .pkl
joblib.dump(vectorizer, 'vectorizer.pkl')

['vectorizer.pkl']